# 深層学習スクラッチ　リカレントニューラルネットワーク

# 【問題1】SimpleRNNのフォワードプロパゲーション実装

SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

$
a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + B\\
h_t = tanh(a_t)
$


a
t
 : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)


h
t
 : 時刻tの状態・出力 (batch_size, n_nodes)


x
t
 : 時刻tの入力 (batch_size, n_features)


W
x
 : 入力に対する重み (n_features, n_nodes)


h
t
−
1
 : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)


W
h
 : 状態に対する重み。 (n_nodes, n_nodes)


B
 : バイアス項 (n_nodes,)

初期状態 
h
0
 は全て0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 
x
 は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。


分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [3]:
np.dot(0,w_h)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [4]:
x.shape

(1, 3, 2)

In [5]:
x[:,0,:]

array([[0.01, 0.02]])

In [6]:
w_x.shape

(2, 4)

In [7]:
w_h.shape

(4, 4)

In [8]:
class Tanh:
    def __init__(self):
        self.A = None
        
    def forward(self, A):
        self.A = A
        return (np.exp(self.A) - np.exp(-self.A))/(np.exp(self.A) + np.exp(-self.A))
    
    def backward(self, dZ):
        return dZ*(1 - self.forward(self.A)**2)

In [9]:
class SimpleRNN:
    def __init__(self,X):
        self.w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
        self.w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
        
        self.b = np.array([1, 1, 1, 1]) 
        
        self.batch_size = x.shape[0] # 1
        self.n_sequences = x.shape[1] # 3
        self.n_features = x.shape[2] # 2
        self.n_nodes = w_x.shape[1] # 4
        
        self.h = np.zeros((self.batch_size, self.n_nodes)) # (batch_size, n_nodes)
        
    def forward(self, X, t):#t=0から

        xt = X[:,t,:]
        print(self.h)
        if t == 0:
            at = np.dot(xt, self.w_x) + self.b
        else:
            at = np.dot(xt, self.w_x) + np.dot(self.h, self.w_h) + self.b

        tanh = Tanh()
        self.h = tanh.forward(at)
        return self.h

In [10]:
class ScratchSimpleRNNClassifier():
    def __init__(self):
        pass
    
    def h_s(self,x):
        self.srnn = SimpleRNN(x)
        self.srnn.forward(x,t=0)
        self.srnn.forward(x,t=1)
        answer = self.srnn.forward(x,t=2)
        return answer
        

In [11]:
shinoda = ScratchSimpleRNNClassifier()

In [12]:
shinoda.h_s(x)

[[0. 0. 0. 0.]]
[[0.76188798 0.76213958 0.76239095 0.76255841]]
[[0.792209   0.8141834  0.83404912 0.84977719]]


array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

In [17]:
srnn = SimpleRNN(x)

In [18]:
srnn.forward(x,t=0)

[[0. 0. 0. 0.]]


array([[0.76188798, 0.76213958, 0.76239095, 0.76255841]])

In [19]:
srnn.forward(x,t=1)

[[0.76188798 0.76213958 0.76239095 0.76255841]]


array([[0.792209  , 0.8141834 , 0.83404912, 0.84977719]])

In [20]:
srnn.forward(x,t=2)

[[0.792209   0.8141834  0.83404912 0.84977719]]


array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

In [ ]:
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)

# 【問題2】小さな配列でのフォワードプロパゲーションの実験

A.上に記載しています

In [ ]:
thi = 'shinoda'